### Compare units that canceled with DDM-like units for this category

####Output: ccm_category_neurons.mat files

In [ ]:
dataPath = fullfile(projectRoot,'data',projectDate,subject);

cEpochList = {'presacc','presaccNoVis','presaccRamp','visPresacc','presaccNoPostsacc','presaccPeak'};
dgEpochList = {'Stim','Sacc','Post'};

cEpochList = {'presaccRamp','presaccPeak'};
cEpochList = {'presacc'};
dgEpochList = {'Stim'};
cancelEpoch = 'presacc_cancel';

cancelType = 'meanSdf'
% cancelType = 'postSsrt'

for i = 1 : length(cEpochList)
    for j = 1 : length(dgEpochList)
        
        dgEpoch = dgEpochList{j};
        cEpoch = cEpochList{i};




        % LOAD ALL RELEVANT DATA AND LISTS

        % Load Classic neuron classifications
        cFileName = fullfile(dataPath, ['ccm_',cEpochList{i},'_neurons', addMulti]);
        c = load(cFileName);
        classic = c.neurons;
        
        % Load Ding and gold (choice) neuron classifications
        dgFileName = fullfile(dataPath, ['ccm_',ddmType, dgEpochList{j}, '_neurons', addMulti]);
        dg = load(dgFileName);
        ddm = dg.neurons;


        % Load Cancel neuron classifications 
        caFileName = fullfile(dataPath, ['ccm_',cancelEpoch,'_',cancelType,'_neurons', addMulti]);
        ca = load(caFileName);
        cancel = ca.neurons;


        
        
        if deleteSessions
            sessionRemove = ccm_exclude_sessions(subject);
            classic = classic(~ismember(classic.sessionID, sessionRemove),:);
            ddm = ddm(~ismember(ddm.sessionID, sessionRemove),:);
            cancel = cancel(~ismember(cancel.sessionID, sessionRemove),:);
        end


        
        
        [~, iCla, iDdm] = setxor(classic, ddm);
        classicDdm = intersect(classic, ddm);
        classicNoDdm = classic(iCla, :);
        ddmNoClassic = ddm(iDdm, :);


        [~, iCla, iCan] = setxor(classic, cancel);
        classicCancel = intersect(classic, cancel);
        classicNoCancel = classic(iCla, :);
        cancelNoClassic = cancel(iCan, :);


        [~, iCan, iDdm] = setxor(cancel, classicDdm);
        classicDdmCancel = intersect(classicDdm, cancel);
        classicCancelNoDdm = cancel(iCan,:);
        classicDdmNoCancel = classicDdm(iDdm,:);

        classicNoDdmNoCancel = intersect(classicNoDdm, classicNoCancel);
        

        % Save the units as population data, to be called later efficiently
        neurons = classicDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'_', ddmType, dgEpochList{j},'_neurons', addMulti]), 'neurons')

        neurons = classicNoDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'_no', ddmType, dgEpochList{j},'_neurons', addMulti]), 'neurons')

        neurons = ddmNoClassic;
        save(fullfile(dataPath, ['ccm_',ddmType,dgEpochList{j},'_no',cEpochList{i},'_neurons', addMulti]), 'neurons')

        neurons = cancelNoClassic;
        save(fullfile(dataPath, ['ccm_cancel_',cancelType,'_no',cEpochList{i},'_neurons', addMulti]), 'neurons')

        neurons = classicDdmCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'_', ddmType, dgEpochList{j},'_cancel_',cancelType,'_neurons', addMulti]), 'neurons')

        neurons = classicCancelNoDdm;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'_cancel_',cancelType,'_no',ddmType,dgEpochList{j},'_neurons', addMulti]), 'neurons')

        neurons = classicDdmNoCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'_',ddmType,dgEpochList{j},'_noCancel_',cancelType,'_neurons', addMulti]), 'neurons')

        neurons = classicNoDdmNoCancel;
        save(fullfile(dataPath, ['ccm_',cEpochList{i},'_no',ddmType,dgEpochList{j},'_noCancel_',cancelType,'_neurons', addMulti]), 'neurons')


    end
end



## Make a table of all the units with their ddm X cancel classification

In [ ]:
        % Load each list and build table
        categoryList = {'presacc_ddmRankMeanStim_cancel_meanSdf','presacc_cancel_meanSdf_noddmRankMeanStim','presacc_ddmRankMeanStim_noCancel_meanSdf','presacc_noddmRankMeanStim_noCancel_meanSdf'};
        categoryString = {'ddm_cancel','noddm_cancel','ddm_nocancel','noddm_nocancel'};

        categoryTable = table();
        for i = 1 : length(categoryList)   
            load(fullfile(dataPath, ['ccm_',categoryList{i},'_neurons',addMulti]))
            iTable = [neurons.sessionID, neurons.unit, cell2table(cellstr(repmat(categoryString{i}, size(neurons, 1), 1)))];
            size(iTable)
            categoryTable = [categoryTable; iTable];
        end
        categoryTable.Properties.VariableNames = {'sessionID', 'unit', 'category'}


### Save the table as a CSV

In [ ]:

writetable(categoryTable, fullfile(dataPath, [subject,'_ddm_cancel_units.csv']))


## Make a table of all the units with their ddm X cancel classification, all cancel data, and all ddm data

In [ ]:
% Load the population choice analysis
epochList = {'Stim'};
fileName = fullfile(dataPath, ['ccm_ddm',epochList{1},'_neuronTypes', addMulti]);
load(fileName)  % Loads neuronTypes table

% load the population of cancel time anlysis
fileName = fullfile(dataPath, 'go_vs_canceled', ssrtUse, ['ccm_canceled_vs_go_neuronTypes', addMulti]);
load(fileName)  % Loads the cancelTypes table

session = [];
unit = [];
category = [];
hemisphere = [];
rf = [];
ssd = [];
ssrt = [];
coherence = [];
easyHard = [];
nStop = [];
pValue40msStopStop = [];
cancelTime2Std = [];
cancelTimeSdf = [];
targToStopDiffSD = [];
preSaccDiffSD = [];

% Loop through the neurons list and fill out a table
for i = 1 : size(categoryTable, 1)
    
    % Find the index in the choice data
    iChoiceInd = strcmp(neuronTypes.sessionID, categoryTable.sessionID(i)) & strcmp(neuronTypes.unit, categoryTable.unit(i));
    
    % Find the index in the cancel data: should be same as choice data index
    iCancelInd = strcmp(cancelTypes.sessionID, categoryTable.sessionID(i)) & strcmp(cancelTypes.unit, categoryTable.unit(i));
    
    % How many conditions per neuron, as per the cancelTypes table data?
    nRep = size(cancelTypes.stopStopSsd{iCancelInd}, 1);
    
    % Data from the categoryTable (ddmXcancel labels)
    session = [session; repmat(categoryTable.sessionID(i), nRep, 1)];
    unit = [unit; repmat(categoryTable.unit(i), nRep, 1)];
    category = [category; repmat(categoryTable.category(i), nRep, 1)];
    
    % Data from the neuronTypes choice data
    hemisphere = [hemisphere; repmat(neuronTypes.hemisphere(iChoiceInd), nRep, 1)];


    % Data from the cancelTypes choice data
    rf = [rf; repmat(cancelTypes.rf(iCancelInd), nRep, 1)];
    ssd = [ssd; cancelTypes.stopStopSsd{iCancelInd}];
    ssrt = [ssrt; cancelTypes.stopStopSsrt{iCancelInd}];
    coherence = [coherence; cancelTypes.stopStopCoh{iCancelInd}];
    easyHard = [easyHard; cancelTypes.stopStopCond{iCancelInd}];
    nStop = [nStop; cancelTypes.nStopStop{iCancelInd}];
    pValue40msStopStop = [pValue40msStopStop; cancelTypes.pValue40msStopStop{iCancelInd}];
    cancelTime2Std = [cancelTime2Std; cancelTypes.cancelTime2Std{iCancelInd}];
    cancelTimeSdf = [cancelTimeSdf; cancelTypes.cancelTimeSdf{iCancelInd}];
    targToStopDiffSD = [targToStopDiffSD; cancelTypes.targToStopDiffSD{iCancelInd}];
    preSaccDiffSD = [preSaccDiffSD; cancelTypes.preSaccDiffSD{iCancelInd}];
   
end

ddmCancelData = table(session, unit, category, hemisphere, rf, ssd, ssrt, coherence, easyHard,...
                      nStop, pValue40msStopStop, cancelTime2Std, cancelTimeSdf, targToStopDiffSD, preSaccDiffSD);



### Save the table as a CSV

In [ ]:

writetable(ddmCancelData, fullfile(dataPath, [subject,'_ddm_cancel_data.csv']))


##Population plots


In [ ]:
categoryList = {'presacc','presacc_ddmStim_cancel_meanDifference', 'presacc_ddmStim_noCancel_meanDifference', 'presacc_noddmStim_noCancel_meanDifference','presacc_cancel_meanDifference_noddmStim',...
               'presacc_ddmRankMeanStim_cancel_meanDifference', 'presacc_ddmRankMeanStim_noCancel_meanDifference', 'presacc_noddmRankMeanStim_noCancel_meanDifference','presacc_cancel_meanDifference_noddmRankMeanStim'};

categoryList = {'presacc','presacc_ddmStim_cancel_trialByTrial', 'presacc_ddmStim_noCancel_trialByTrial', 'presacc_noddmStim_noCancel_trialByTrial','presacc_cancel_trialByTrial_noddmStim',...
               'presacc_ddmRankMeanStim_cancel_trialByTrial', 'presacc_ddmRankMeanStim_noCancel_trialByTrial', 'presacc_noddmRankMeanStim_noCancel_trialByTrial','presacc_cancel_trialByTrial_noddmRankMeanStim'};
categoryList = {'presacc_ddmRankMeanStim_cancel_trialByTrial'};



categoryList = {'presacc_ddmRankMeanStim_cancel_meanSdf','presacc_noddmRankMeanStim_noCancel_meanSdf'};
categoryList = {'presacc_cancel_meanSdf_noddmRankMeanStim','presacc_ddmRankMeanStim_noCancel_meanSdf'};
% categoryList = {'presacc_ddmRankMeanStim_cancel_meanSdf','presacc_cancel_meanSdf_noddmRankMeanStim','presacc_ddmRankMeanStim_noCancel_meanSdf','presacc_noddmRankMeanStim_noCancel_meanSdf'};
categoryList = {'presacc_ddmRankMeanStim_cancel_meanSdf'};


opt = ccm_population_neuron_plot;

opt.doGos = false;
opt.doCanceled = true;
opt.doNoncanceled = false;
opt.easyOnly = false; 
opt.inOnly = true;
opt.plotSEM = false;

opt.multiUnit = multiUnit;
opt.normalize = true;
opt.excludeSessions = deleteSessions;
opt.ssrtUse = 'intWeightPerSession';
opt.saccadeBaseRatio = [];

    opt.categoryName = categoryList;
    ccm_population_neuron_plot(subject,projectRoot,projectDate,opt);

    
%for i = 1 : length(categoryList)    
  %  opt.categoryName = categoryList{i};
 %   ccm_population_neuron_plot(subject,projectRoot,projectDate,opt);
%end


### Run stats on the population

In [ ]:
categoryList = {'presacc'};
categoryList = {'presacc_ddmStim_cancel'};
categoryList = {'presacc_ddmStim'};
categoryList = {'presacc','presacc_cancel', 'presacc_noCancel', 'presacc_ddmStim', 'presacc_noddmStim', 'presacc_ddmStim_cancel', 'presacc_cancel_noddmStim', 'presacc_ddmStim_noCancel', 'presacc_noddmStim_noCancel', };


opt = ccm_population_neuron_plot;

opt.doStops = true;
opt.multiUnit = multiUnit;
opt.normalize = normalizeData;
opt.excludeSessions = deleteSessions;
opt.ms2Std  = 75;
opt.ssrtUse = 'intWeightPerSession';
opt.saccadeBaseRatio = [];

for i = 1 : length(categoryList)
    
    opt.categoryName = categoryList{i};

    ccm_population_neuron_stats(subject,projectRoot,projectDate,opt);
    
end


#Continue to Summary Analyses Notebook